In [2]:
# Import libraries
# Ignore warnings
import numpy as np
from scipy.stats import norm
import numpy as np
import astropy
import astropy_healpix
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import SkyCoord
import healpy as hp
import pandas as pd
import math
import scipy
import statistics as st
from scipy import stats
import matplotlib
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from IPython.display import Image

%matplotlib inline

NSIDE=2048
Npix=12*NSIDE**2

pi=math.pi

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Read in AMF catalog
dr9 = pd.read_csv('Data/dr9_main_complete.csv', header=-1)
dr9.columns = ['amf_no','amf_ra','amf_dec','amf_z','amf_lk','amf_rh','amf_r200','amf_rc','amf_c','amf_str_rh','bax_id','mcxc_id','red_id','whl_id']

In [4]:
# Read in Burenin catalog
bur = pd.read_csv('Data/planck_extended_burenin.csv')

In [5]:
# Read in Redmapper cat1alog
red = pd.read_csv("Data/redmapper_v6.3_errs.csv",header=-1)
red.columns = ['red_no','red_ra','red_dec','red_z','red_zerr','red_rh','red_rherr']

In [6]:
# Read in WHL catalog
whl = pd.read_fwf('Data/whl2_v2.dat')
whl.columns = ['whl_no','whl_id','RA','dec','z','rich']

In [7]:
# Read in Planck catalog
plsz2 = pd.read_csv('Data/planck_sz2.csv',header=-1)
plsz2.columns = ['pl_no','pl_name','pl_glon','pl_glat','pl_ra','pl_dec','pl_poserr','pl_sn',\
                   'pl_pipe','pl_pipedet','pl_pccs2','pl_psz','pl_irflag','pl_qneural','pl_y5r500',\
                   'pl_y5r500err','pl_valid','pl_zid','pl_z','pl_msz','pl_mszerrup','pl_mszerrlow',\
                   'pl_mcxc','pl_red','pl_act','pl_spt','pl_wise','pl_ami','pl_cosmo','pl_comment']

In [8]:
# Read in ymaps, stddevs (in Galactic coordinates)
nilc_ymap = hp.read_map('Data/COM_CompMap_Compton-SZMap-nilc-ymaps_2048_R2.00.fits')
nilc_stddevmap = hp.read_map('Data/COM_CompMap_Compton-SZMap-nilc-stddev_2048_R2.00.fits')

NSIDE = 2048
ORDERING = RING in fits file
INDXSCHM = IMPLICIT
NSIDE = 2048
ORDERING = RING in fits file
INDXSCHM = IMPLICIT


In [9]:
# Convert the RA and DEC to galactic coordinates
c = SkyCoord(dr9['amf_ra'],dr9['amf_dec'],frame='icrs',unit='deg')
dr9['glat'] = c.galactic.b.degree
dr9['glon'] = c.galactic.l.degree

c=SkyCoord(bur['bur_ra'],bur['bur_dec'],frame='icrs',unit='deg')
bur['glat'] = c.galactic.b.degree # Galactic Latitude
bur['glon'] = c.galactic.l.degree # Galactic Longitude

In [ ]:
# Smooth noise map by 1 degree
# The essential difference between Burenin and this, is that he uses
# a 1 degree radius median filter for smoothing (not available on healpy)
stddev_smoothed = hp.smoothing(nilc_stddevmap,fwhm=np.radians(1.))

# Calculate signal over noise (smoothed)
sn_smoothed_n = nilc_ymap/stddev_smoothed

# Smooth the sn_smoothed_n map by 1 degree
sn_smoothed_n_smoothed = hp.smoothing(sn_smoothed_n,fwhm=np.radians(1.))

# Subtract the smoothed signal to noise map from the signal to noise map
sn_result = sn_smoothed_n - sn_smoothed_n_smoothed

In [ ]:
# Mask the sn_result array
# By convention the mask is 0 where the data is masked, while numpy defines data masked when 
# the mask is true, so it is necessary to flip the mask.
mask = hp.read_map('Data/maskplanckfullsz61.fits')
sn_masked = hp.ma(sn_result,badval=+1.6375e+30)
sn_masked2 = hp.ma(sn_result)
sn_masked2.mask = np.logical_not(mask)
sn_masked.mask = np.logical_not(mask)

In [ ]:
scipy.stats.describe(mask)

In [ ]:
# Plot the masked array
hp.mollview(sn_masked,norm='hist',coord=['G'],title='Masked S/N map')
Image(filename='Papers/burfig.png') 

In [ ]:
# Histogram of the above map
plt.figure(figsize=(24,10))
plt.hist(sn_masked.compressed(), bins=500,normed=True,fill=False,edgecolor='k');
plt.xlim((-9,14))
plt.ylim((4e-5,1))
plt.yscale('log');
plt.xticks(np.arange(-9, 14, 1.0))
plt.xlabel('S/N');
plt.title('Signal to Noise histogram of masked array')

ln = np.linspace(-9, 14, len(sn_masked))
g = stats.norm.pdf(ln, 0, 1.58)  
plt.plot(ln, g, label="Norm", color="red")

plt.show()

In [ ]:
d_glonR = []
d_glatR = []
d_glon = []
d_glat = []
d_raR = []
d_decR = []
d_rh = []
d_sn = []
i = 0

for i in range(dr9.shape[0]):
        d_raR.append("%.1f" % dr9.iloc[i,1])
        d_decR.append("%.1f" % dr9.iloc[i,2])
        d_glon.append(dr9.iloc[i,15])
        d_glat.append(dr9.iloc[i,14])
        d_glonR.append("%.1f" % dr9.iloc[i,15])
        d_glatR.append("%.1f" % dr9.iloc[i,14])
        d_pix = hp.ang2pix(2048,dr9.iloc[i,15],dr9.iloc[i,14],lonlat=True)
        d_rh.append(dr9.iloc[i,5])
        d_sn.append(sn_masked[d_pix])

tup1 = zip(d_glon, d_glat)

amf_round = pd.DataFrame({'Glon': d_glonR, 'Glat': d_glatR, 'RA': d_raR, 'dec': d_decR})

In [ ]:
d_glonR1 = []
d_glatR1 = []
d_glon1 = []
d_glat1 = []
d_rh1 = []

for i in range(dr9.shape[0]):
        d_glon1.append(dr9.iloc[i,15])
        d_glat1.append(dr9.iloc[i,14])
        d_glonR1.append("%.1f" % dr9.iloc[i,15])
        d_glatR1.append("%.1f" % dr9.iloc[i,14])

tup1 = zip(d_glon1, d_glat1)
  
df11 = pd.DataFrame({'Glon': d_glon1, 'Glat': d_glat1})
df21 = pd.DataFrame({'Glon': d_glonR1, 'Glat': d_glatR1})

b_glon1 = []
b_glat1 = []
b_rh1 = []

for i in range(bur.shape[0]):
        b_glon1.append("%.1f" % bur.iloc[i,6])
        b_glat1.append("%.1f" % bur.iloc[i,5])

tup2 = zip(b_glon1, b_glat1)

df31 = pd.DataFrame({'Glon': b_glon1, 'Glat': b_glat1})

amfset1 = set(list(zip(d_glonR1, d_glatR1)))
burset1 = set(list(zip(b_glon1, b_glat1)))
amfNOTbur1 = amfset1-burset1
aNOTb_lon1,aNOTb_lat1, = zip(*amfNOTbur1)
len(amfNOTbur1)

new1 = pd.DataFrame({'Glon': aNOTb_lon1, 'Glat': aNOTb_lat1})

In [ ]:
amf_all = pd.DataFrame({'Glon': d_glon, 'Glat': d_glat, 'Rh': d_rh, 'SN': d_sn})

amfall_sns = []
amfall_sns1 = []

for i in range(amf_all.shape[0]):
    if amf_all.iloc[i,3]:
        amfall_sns.append(amf_all.iloc[i,3])
        amfall_sns1.append((amf_all.iloc[i,3]*1.65)-1.15)
    else:
        amfall_sns.append(0)
        amfall_sns1.append(0)

plt.hist(amfall_sns,bins=492,normed=True,fill=False,edgecolor='blue',alpha=.6);
plt.xlim((-9,14))
plt.ylim(1e-4,)
plt.yscale('log')
plt.xlabel('S/N')

plt.show()

In [ ]:
amfall_sns

In [ ]:
amfunique_sns = []

for i in range(new1.shape[0]):
    pix = hp.ang2pix(2048,float(new1.iloc[i,0]),float(new1.iloc[i,1]),lonlat=True)
    if sn_masked[pix]:
        amfunique_sns.append(sn_masked[pix])
    else:
        amfunique_sns.append(0)

plt.hist(amfunique_sns,bins=492,normed=True,fill=False,edgecolor='blue',alpha=.6);
plt.xlim((-9,14))
plt.ylim(1e-4,)
plt.yscale('log')
plt.xlabel('S/N')

plt.show()

In [ ]:
plt.hist(amfunique_sns,bins=500,normed=True,fill=False,edgecolor='blue',alpha=0.6);
plt.hist(sn_masked.compressed(), bins=500,normed=True,fill=False,edgecolor='r',alpha= 0.6);

plt.xlim((-9,14))
plt.ylim(1e-5,)
plt.yscale('log');
plt.xlabel('S/N');
plt.title('Signal to Noise histogram of masked array');

# ln = np.linspace(-9, 14, len(sn_masked))
# g = stats.norm.pdf(ln, 0, 1.73)  
# plt.plot(ln, g, label="Norm", color="red")

plt.show()

In [ ]:
# Histogram of the above map
plt.figure(figsize=(12,12))
plt.hist(plsz2['pl_sn'], bins=400,normed=True,fill=False,edgecolor='red', alpha=0.6);
plt.hist(amfall_sns1, bins=400,normed=True,fill=False,edgecolor='blue', alpha=0.5);
# plt.hist(sn_masked.compressed(), bins=400,normed=True,fill=False,edgecolor='yellow', alpha=0.4);
plt.yscale('log')
plt.xlabel('SN')
plt.show()

In [ ]:
plnk = pd.read_csv('Data/file1.csv')
plnk.sort_values(by=['amf_no'])
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
plnk

In [ ]:
plt.figure(figsize=(20,10))
plt.scatter(np.log10(plnk['pl_sn_x']),plnk['delta_rh'], c = plnk['amf_z'])
plt.colorbar()